# Generación dataset diagnosticos

## Extraer diagnosticos ICD-10 a partir del XML oficial de MIMIC-IV

In [ ]:
import xml.etree.ElementTree as ET
import csv
import re

def remove_dots(code: str) -> str:
    """
    Eliminar los puntos de un código ICD, e.g. 'A00.1' -> 'A001'.
    """
    return code.replace('.', '')

def extract_domain_code(icd_code_no_dots: str) -> str:
    """
    Devolver los primeros 3 caracteres de un código sin puntos.
    E.g. 'A001' -> 'A00'.
    """
    return icd_code_no_dots[:3]

def clean_section_desc(desc: str) -> str:
    """
    Eliminar cualquier texto entre paréntesis al final de la descripción.
    E.g. 'Intestinal infectious diseases (A00-A09)' -> 'Intestinal infectious diseases'.
    """
    if not desc:
        return desc
    # Buscar "cualquier caracter" al final y eliminar junto con espacios adyacentes
    return re.sub(r'\s*\([^()]*\)\s*$', '', desc).strip()

def parse_diag(diag_element, rows, domain_desc):
    """
    Procesar recursivamente un <diag> (y sus hijos <diag>), añadiendo filas al CSV.
    
    - diag_element: Elemento XML <diag> actual.
    - rows: lista [icd_code, icd_domain_code, domain_description].
    - domain_desc: descripción limpia de la <section> (sin paréntesis).
    """
    current_name = diag_element.findtext('name')
    if not current_name:
        return  # Si no hay <name>, no es un diag con código

    # Código completo sin puntos
    icd_code_no_dots = remove_dots(current_name)
    # Codigo del dominio: 3 primeros caracteres
    domain_code = extract_domain_code(icd_code_no_dots)

    # Agregar la fila para este diag
    rows.append([
        icd_code_no_dots,  # icd_code
        domain_code,       # icd_domain_code
        domain_desc        # domain_description (limpio de la <section>)
    ])

    # Procesar hijos <diag>
    for child_diag in diag_element.findall('diag'):
        parse_diag(child_diag, rows, domain_desc)

def main():
    input_file = "icd-10-cm-tabular-2025.xml"
    output_file = "icd-10-cm-tabular-2025.csv"

    tree = ET.parse(input_file)
    root = tree.getroot()  # <ICD10CM.tabular>
    
    rows = []

    # Recorrer <chapter> y luego <section>
    for chapter in root.findall('chapter'):
        for section in chapter.findall('section'):
            # Descripción de la sección
            original_desc = section.findtext('desc', default="")
            # Eliminar paréntesis
            cleaned_desc = clean_section_desc(original_desc)

            # Parsear recurrsivamente para cada <diag> top-level de la <section>.
            for diag_element in section.findall('diag'):
                parse_diag(diag_element, rows, cleaned_desc)

    # Guardar resultados como CSV
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["icd_code", "icd_domain_code", "domain_description"])
        writer.writerows(rows)

    print(f"CSV generado: {output_file}")

if __name__ == "__main__":
    main()